<a href="https://colab.research.google.com/github/chirazedrine/EmoGen/blob/main/EmoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0: Dowload dependencies

In [ ]:
!pip install torch torchvision

# Step 1: Download and extract the dataset

In [3]:
!wget -O EmoSet-118K.zip "https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv"


--2024-03-20 14:59:11--  https://www.dropbox.com/scl/fi/myue506itjfc06m7svdw6/EmoSet-118K.zip?dl=1&rlkey=7f3oyjkr6zyndf0gau7t140rv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc63266babafe081a32018c9ce72.dl.dropboxusercontent.com/cd/0/inline/CPfA6g5GWkVXmWyUv6hjlGqSibUyyodWGB0UuYQlQYUGG9f5conzAJX2569mFyzUWmidqJs-ID8nUlH_6j1NLUGZgtDb_D-uz_OCMb3cA6Qhzwzq0tfEiT_vQ0N7jrIwc_9l8gDiasRND6UNMtzoMny4/file?dl=1# [following]
--2024-03-20 14:59:12--  https://uc63266babafe081a32018c9ce72.dl.dropboxusercontent.com/cd/0/inline/CPfA6g5GWkVXmWyUv6hjlGqSibUyyodWGB0UuYQlQYUGG9f5conzAJX2569mFyzUWmidqJs-ID8nUlH_6j1NLUGZgtDb_D-uz_OCMb3cA6Qhzwzq0tfEiT_vQ0N7jrIwc_9l8gDiasRND6UNMtzoMny4/file?dl=1
Resolving uc63266babafe081a32018c9ce72.dl.dropboxusercontent.com (uc63266babafe081a32018c9ce72.dl.dropboxusercontent.com)...

In [4]:
!unzip -q EmoSet-118K.zip -d /content/EmoSet-118K/

# Step 2: Parse the JSON files to create a mapping for image paths and labels

In [5]:
import os
import json
import torch
import torchvision.models as models
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn

base_path = '/content/EmoSet-118K'
annotations_dir = os.path.join(base_path, 'annotation')
images_dir = os.path.join(base_path, 'image')

# Parse JSON annotations
def parse_annotations(annotations_dir):
    annotations = []
    for emotion_dir_name in os.listdir(annotations_dir):
        emotion_dir_path = os.path.join(annotations_dir, emotion_dir_name)
        if os.path.isdir(emotion_dir_path):
            for annotation_file in os.listdir(emotion_dir_path):
                annotation_file_path = os.path.join(emotion_dir_path, annotation_file)
                with open(annotation_file_path, 'r') as f:
                    annotation_data = json.load(f)
                    annotations.append(annotation_data)
    return annotations

annotations = parse_annotations(annotations_dir)

# Verify the number of annotations loaded
print(f"Loaded {len(annotations)} annotations.")

# Extract unique emotions and create a mapping to indices
unique_emotions = sorted({anno['emotion'] for anno in annotations})
emotion_to_idx = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
print(f"Unique emotions found: {list(emotion_to_idx.keys())}")

class EmoSetDataset(Dataset):
    def __init__(self, annotations, images_dir, transform=None):
        self.annotations = annotations
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        image_path = os.path.join(self.images_dir, annotation['emotion'], f"{annotation['image_id']}.jpg")
        image = Image.open(image_path).convert('RGB')
        label = emotion_to_idx[annotation['emotion']]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Dataset and DataLoader
dataset = EmoSetDataset(annotations, images_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


Loaded 118102 annotations.
Unique emotions found: ['amusement', 'anger', 'awe', 'contentment', 'disgust', 'excitement', 'fear', 'sadness']


# Step 3: Define the CNN model for emotion classification

In [7]:
import torch
import torchvision.models as models
from torchvision.models import ResNet50_Weights

# Load pre-trained ResNet-50 model
weights = ResNet50_Weights.DEFAULT
model = models.resnet50(weights=weights)

# Modify final layer to match the number of unique emotions
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(unique_emotions))

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 73.8MB/s]


# Step 4: Define loss function and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Check if the data loader can fetch data
for images, labels in data_loader:
    print(f"Fetched {images.shape[0]} samples.")
    break  # Only fetch one batch for checking


Fetched 32 samples.


# Step 5: Train the model

In [9]:
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm

# Setup device: Use GPU if available, else fall back to CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

model = model.to(device)

num_epochs = 5

# Start the training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Wrap your data loader with tqdm for a progress bar
    data_loader_tqdm = tqdm(data_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True)

    for inputs, labels in data_loader_tqdm:
        # Move inputs and labels to the correct device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass, backward pass, and optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Calculate and accumulate loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct_predictions += torch.sum(preds == labels.data)
        total_predictions += labels.size(0)

        # Update the progress bar with the current loss and accuracy
        data_loader_tqdm.set_description(f'Epoch {epoch+1}/{num_epochs} Loss: {running_loss/total_predictions:.4f} Acc: {correct_predictions.double()/total_predictions:.4f}')

    # Log the metrics for the epoch
    print(f'Finished Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/total_predictions:.4f}, Accuracy: {correct_predictions.double()/total_predictions:.4f}')

print('Finished Training')

Using device: cuda


Epoch 1/5 Loss: 1.2483 Acc: 0.5477:  46%|████▌     | 1693/3691 [09:01<10:50,  3.07it/s]

# Step 6: Evaluate the model

In [ ]:
import torch
model_path = '/content/drive/My Drive/emo_model.pth'
torch.save(model.state_dict(), model_path)

# Reload the model weights
model.load_state_dict(torch.load(model_path))

# Evaluate on the test set
test_annotations = [anno for anno in annotations.values() if anno['split'] == 'test']
test_dataset = EmoSetDataset(test_annotations, images_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()
running_corrects = 0

for inputs, labels in test_loader:
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    running_corrects += torch.sum(preds == labels.data)

test_acc = running_corrects.double() / len(test_loader.dataset)
print(f'Test Accuracy: {test_acc:.4f}')

NameError: name 'model' is not defined